In [1]:
pip install tensorflow tensorflow_hub tensorflow_datasets


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [5]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/cancer/
  inflating: dataset/cancer/000000 (6).png  
  inflating: dataset/cancer/000002 (4).png  
  inflating: dataset/cancer/000002 (6).png  
  inflating: dataset/cancer/000002.png  
  inflating: dataset/cancer/000003 (3).png  
  inflating: dataset/cancer/000003 (4).png  
  inflating: dataset/cancer/000003.png  
  inflating: dataset/cancer/000004 (4).png  
  inflating: dataset/cancer/000004 (5).png  
  inflating: dataset/cancer/000004 (7).png  
  inflating: dataset/cancer/000005 (3).png  
  inflating: dataset/cancer/000005 (9).png  
  inflating: dataset/cancer/000006 (4).png  
  inflating: dataset/cancer/000006.png  
  inflating: dataset/cancer/000007 (4).png  
  inflating: dataset/cancer/000007.png  
  inflating: dataset/cancer/000008 (10).png  
  inflating: dataset/cancer/000008 (3).png  
  inflating: dataset/cancer/000009 (2).png  
  inflating: dataset/cancer/000009 (3).png  
  inflating: dataset/cancer/000009 (4).png

In [6]:
# Data preprocessing
img_height, img_width = 224, 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 996 files belonging to 2 classes.
Using 797 files for training.
Found 996 files belonging to 2 classes.
Using 199 files for validation.


In [7]:
# Data augmentation
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [8]:
# Model creation
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

inputs = keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
classification_output = layers.Dense(1, activation='sigmoid', name='classification')(x)

model = keras.Model(inputs, classification_output)

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
# Train the model
epochs = 20
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/20
25/25 [==============================] - 4s 122ms/step - loss: 0.0835 - accuracy: 0.9711 - val_loss: 0.0697 - val_accuracy: 0.9749
Epoch 2/20
25/25 [==============================] - 4s 121ms/step - loss: 0.0726 - accuracy: 0.9774 - val_loss: 0.0692 - val_accuracy: 0.9648
Epoch 3/20
25/25 [==============================] - 5s 178ms/step - loss: 0.0841 - accuracy: 0.9661 - val_loss: 0.0724 - val_accuracy: 0.9598
Epoch 4/20
25/25 [==============================] - 4s 120ms/step - loss: 0.0768 - accuracy: 0.9649 - val_loss: 0.0763 - val_accuracy: 0.9648
Epoch 5/20
25/25 [==============================] - 4s 124ms/step - loss: 0.0713 - accuracy: 0.9774 - val_loss: 0.0663 - val_accuracy: 0.9648
Epoch 6/20
25/25 [==============================] - 5s 171ms/step - loss: 0.0686 - accuracy: 0.9762 - val_loss: 0.0625 - val_accuracy: 0.9698
Epoch 7/20
25/25 [==============================] - 4s 120ms/step - loss: 0.0712 - accuracy: 0.9749 - val_loss: 0.0684 - val_accuracy: 0.9648
Epoch 

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 classification (Dense)      (None, 1)                 1281  

In [13]:
# Save the model
model.save('cancer_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
model.save('cancer_detection_model.keras')

In [15]:
model = tf.keras.models.load_model('cancer_detection_model.keras')

In [16]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


In [17]:
# Save the model
with open('cancer_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [18]:
from google.colab import files
files.download('cancer_detection_model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image

In [54]:
# Function to get image array
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

# Function to make Grad-CAM heatmap
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations of the last conv layer
    last_conv_layer_model = tf.keras.Model(model.inputs, model.get_layer(last_conv_layer_name).output)

    # Create a model that maps the activations of the last conv layer to the final class predictions
    classifier_input = keras.Input(shape=model.get_layer(last_conv_layer_name).output.shape[1:])
    x = classifier_input
    for layer_name in [layer.name for layer in model.layers if layer.name not in [last_conv_layer_name]]:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Compute the gradient of the top predicted class with respect to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Pool the gradients over all the axes leaving out the channel dimension
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by 'how important this channel is' with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purposes, normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [55]:
# Load model
model = tf.keras.models.load_model('cancer_detection_model.keras')

In [67]:
# Print base model summary to find the last conv layer
base_model = model.get_layer('mobilenetv2_1.00_224')
base_model.summary()

# Last conv layer is 'Conv_1' within 'mobilenetv2_1.00_224'
last_conv_layer_name = 'Conv_1'


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        multiple                     0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [57]:
# Prepare image
img_path = '000117.png'
img_array = get_img_array(img_path, size=(224, 224))

In [70]:
# Resize the image array to the expected input shape of the convolutional layer
img_array_resized = tf.image.resize(img_array, (7, 7))

# Generate heatmap
heatmap = make_gradcam_heatmap(img_array_resized, base_model, last_conv_layer_name)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


ValueError: Exception encountered when calling layer "Conv1" (type Conv2D).

Depth of input (1280) is not a multiple of input depth of filter (3) for '{{node Conv1/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true](Placeholder, Conv1/Conv2D/ReadVariableOp)' with input shapes: [?,7,7,1280], [3,3,3,32].

Call arguments received by layer "Conv1" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 7, 7, 1280), dtype=float32)

In [46]:
# Display heatmap
plt.matshow(heatmap)
plt.show()

NameError: name 'heatmap' is not defined

In [ ]:
# Superimpose the heatmap on the original image
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)
    jet = plt.cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    superimposed_img.save(cam_path)

    display(Image.open(cam_path))

# Superimpose and save the image with Grad-CAM
save_and_display_gradcam(img_path, heatmap)